# PostHog-LLM 

PostHog-LLM is an extended version of PostHog to work with text analytics and LLM applications. We integrated machine learning models during data ingestion that extract text features from your LLM interactions and we also introduce the ability to visualize your LLM interactions in the insights page!

Head over to our [GitHub](https://github.com/postlang/posthog-llm) to quickly deploy your instance locally

This demo shows a simple example of how to interact with a chat model and send the interactions to PostHog-LLM for analysis.

## Install the necessary packages

In [ ]:
# restart notebook
!pip install openai posthog -q

In [ ]:
import posthog

posthog.api_key = "" # in your PostHog instance under 'Settings' -> 'Project API Key'
posthog.host = "" # e.g. http://localhost:8000
open_ai_key = "" # in your OpenAI account under 'API Keys'

## Chat with ChatGPT

We represent each interaction as task!

There's two methods to send your LLM interactions: 

- linking tasks via a “$session_id” property
- incorporating the conversation history in ChatML format within the “$llm_input” property for stateless tasks. (the example below)

In [ ]:
import random
import string
import openai

def random_string(length):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))


def chat_loop(client):
    user_id = random_string(10)
    print("Welcome to the Chat with GPT-3.5 Turbo! Type 'quit' to exit.")

    messages = []

    while True:
        user_message = input("You: ")
        
        if user_message.lower() == "quit":
            print("Exiting the chat. Goodbye!")
            break
        
        messages.append({"role": "user", "content": user_message})
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        print("User: ", user_message)
        ai_message = response.choices[0].message.content
        print(f"GPT-3.5 Turbo: {ai_message}")

        posthog.capture(
            distinct_id=user_id,
            event="llm-task",
            properties={
                '$llm_input': messages,
                '$llm_output': ai_message
                # timestamp is automatically added :)
                # Add any other properties you want to track
            }
        )

        messages.append({"role": "system", "content": ai_message})


client = openai.OpenAI(api_key=open_ai_key)
chat_loop(client)

Check out your Activity feed in your PostHog instance to visualize the interactions you just tracked! 🤗